In [ ]:
import itertools
import math
import re
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import (InsetPosition, inset_axes,
                                                  mark_inset)

plt.rcParams["figure.figsize"] = (16, 6)
# plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams["figure.constrained_layout.use"] = True  # Experimental

sns.set_style("white")
sns.set_context("notebook")

In [ ]:
# Load demand data
demand_df = pd.read_csv(snakemake.input["domestic_demand"], sep=";", index_col="region")
demand_df = demand_df.rename(columns={"demand [GWh]": "demand [TWh]"}) / 1e3

# exporter colors from config
exp_colors = snakemake.config["plotting"]["exporter_colors"]

## Prepare canvas
fig = plt.figure(figsize=(8, 6))
ax = plt.gca()

# Create an inset axis for drawing the full supply curves
axins = inset_axes(ax, height="30%", width="60%", loc="upper left", borderpad=3)

for fn in snakemake.input["supply_curves"]:
    # Load supply curve data
    df = pd.read_csv(fn)
    df["cumulative generation"] /= 1e6  # in TWh

    # Determine region of export for supply curve from filename
    exp = Path(fn).parts[-2].replace("-DE", "")

    # Include demand as black dotted line
    demand = demand_df.loc[exp, "demand [TWh]"]
    df_lower = df[df["cumulative generation"] <= demand]
    df_upper = df[df["cumulative generation"] > demand]
    if not df_lower.empty:
        # Interpolate lcoe/ ylocation
        y_demand = np.interp(
            [demand],
            [
                df_lower.iloc[-1]["cumulative generation"],
                df_upper.iloc[0]["cumulative generation"],
            ],
            [df_lower.iloc[-1]["lcoe"], df_upper.iloc[0]["lcoe"]],
        )
        df_lower = df_lower.append(
            {"lcoe": y_demand[0], "cumulative generation": demand}, ignore_index=True
        ).sort_values("cumulative generation")
        df_upper = df_upper.append(
            {"lcoe": y_demand[0], "cumulative generation": demand}, ignore_index=True
        ).sort_values("cumulative generation")

        ax.plot(
            df_lower["cumulative generation"], df_lower["lcoe"], "--", ms=2, color="k"
        )
        axins.plot(
            df_lower["cumulative generation"], df_lower["lcoe"], "--", ms=5, color="k"
        )

    ax.plot(
        df_upper["cumulative generation"],
        df_upper["lcoe"],
        "-o",
        ms=2,
        color=exp_colors[exp],
        label=exp,
    )
    axins.plot(
        df_upper["cumulative generation"],
        df_upper["lcoe"],
        "-",
        ms=5,
        color=exp_colors[exp],
        label=exp,
    )

ax.legend()
ax.set_xlim(1, 2.5e3)
ax.set_ylim(0, 125)
ax.set_ylabel("Levelised cost of electricity [EUR/MWh]")
ax.set_xlabel("Annual electricity potential [TWh]")

axins.set_ylim(0, 200)
# axins.set_xlabel("[TWh]")
# axins.set_ylabel("[EUR/MWh]")

# Grid lines / style
ax.grid(which="major", axis="both", visible=True, lw=1)
axins.grid(which="major", axis="both", visible=True, lw=1)

# Save figure for all file types (PDF, PNG, ...)
for fp in snakemake.output["figures"]:
    plt.savefig(fp, dpi=snakemake.config["plotting"]["dpi"])